In [ ]:
pip install transformers datasets torch accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/MIT Academics/SEM-8/FINAL YEAR PROJECT/Dataset/Predex_Dataset/train.csv")  # Update with your actual file path
df = df.head(100)  # Remove missing values
df = df.dropna()  # Remove missing values

# Extract columns
texts = df["Input"].tolist()  # Case details
labels = df["Label"].tolist()  # Case outcome (0 or 1)

# Tokenizer for Longformer
longformer_tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

# Tokenize text (truncate long cases to 4096 max tokens)
def tokenize_text(text):
    return longformer_tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=4096,
        return_tensors="pt"
    )

# Tokenize all inputs
tokenized_texts = [tokenize_text(text) for text in texts]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [2]:
from transformers import AutoModel

# Load pretrained Longformer
longformer_model = AutoModel.from_pretrained("allenai/longformer-base-4096")
longformer_model.eval()  # Set to evaluation mode

# Function to extract embeddings
def extract_embeddings(tokenized_input):
    with torch.no_grad():  # Disable gradient computation
        output = longformer_model(**tokenized_input)
    return output.last_hidden_state.mean(dim=1)  # Pooling to get a fixed-size vector

# Extract embeddings for all cases
embeddings = [extract_embeddings(t) for t in tokenized_texts]

# Convert to tensor
embeddings_tensor = torch.cat(embeddings, dim=0)
labels_tensor = torch.tensor(labels)


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/597M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load LegalBERT
legalbert_model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./legalbert-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Define Trainer
trainer = Trainer(
    model=legalbert_model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(embeddings_tensor, labels_tensor),
)

# Start training
trainer.train()
